This notebook aims to do exactly the same as Basic_models.ipynb did, i.e. try to fit the data given to different models with suitable hyperparameters.
However, this time we will transform the data given and see on which of the models it works the best and if it wins 
with the previous results found.

### 1. Setting up functions from Basic_models

In [1]:
import numpy as np 
from numpy.linalg import inv
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from itertools import chain
from scipy.optimize import minimize 
from scipy import stats


df = pd.read_excel('data_clean.xlsx')
df = df.drop(columns = ['Unnamed: 0'])
X = df.iloc[:, 1:7]
y = df['wynik']
print(df.shape)
df.head()

(87, 8)


,rocznik,sesja I,X,XII,sesja II,II,III,wynik
0,19,77.0,53.0,63.0,75.0,75.0,61.0,60
1,19,85.0,69.0,97.0,93.0,91.0,88.0,93
2,19,41.0,31.0,33.0,48.0,66.0,36.0,50
3,19,78.0,81.0,83.0,70.0,94.0,91.0,93
4,19,26.0,22.0,53.0,25.0,41.0,21.0,27


Before we begin there is one thing to bear in mind - we are doing here the same thing as in Basic_models.ipynb but in functions so that we can later transform our data (e.g. using percentiles) and do the same to them.
However! We will want to first tranform our original data $X$ into some $X_{mapped}$, then apply our models to them
getting predictions $y_{mapped\_pred}$, then apply the inverse function to get actual predictions $y_{pred}$ on $X$
itself so that at the end of the day, we can measure the error and compare to the original one.

These are the reasons for additional y_yr_true, type and distr variables in functions - to indicate whether we are dealing with percentile scenario and if so, how should we invert it.

In [2]:
def initialize_cv_split(data, X, y):
    #initializing cross-val split
    params = {} #output
    X_yr, y_yr = {}, {} #dev sets of X and y from a given year
    X_slice, y_slice = {}, {} #training sets complimentary to X_yr and y_yr
    X_slice_trimmed, X_yr_trimmed = {}, {} #the same as above but restricted to the most important columns 
    X_slice_scaled, X_yr_scaled, X_slice_trimmed_scaled, X_yr_trimmed_scaled = {}, {}, {}, {} #scaled versions
    
    
    scaler = StandardScaler()
    
    for year in range(19,23):
        X_yr[year], y_yr[year] = X[data['rocznik'] == year], y[data['rocznik'] == year]
        X_slice[year], y_slice[year] = X[data['rocznik'] != year], y[data['rocznik'] != year]
        X_slice_trimmed[year], X_yr_trimmed[year] = X_slice[year].iloc[:,[0,4,5]], X_yr[year].iloc[:,[0,4,5]]
        
        #scaling normal data
        X_slice_scaled[year] = scaler.fit_transform(X_slice[year])
        X_yr_scaled[year] = scaler.transform(X_yr[year])
        
        #scaling trimmed versions
        X_slice_trimmed_scaled[year] = scaler.fit_transform(X_slice_trimmed[year])
        X_yr_trimmed_scaled[year] = scaler.transform(X_yr_trimmed[year])
        
    #saving all our data to params dictionary
    params['X_yr'], params['y_yr'] = X_yr, y_yr
    params['X_slice'], params['y_slice'] = X_slice, y_slice
    params['X_slice_trimmed'], params['X_yr_trimmed'] = X_slice_trimmed, X_yr_trimmed
    params['X_slice_scaled'], params['X_yr_scaled'], params['X_slice_trimmed_scaled'], params['X_yr_trimmed_scaled'] = \
    X_slice_scaled, X_yr_scaled, X_slice_trimmed_scaled, X_yr_trimmed_scaled
    
        
    return params

In [3]:
def fun(lambda_, Ainv, b):   
    x = b - lambda_
    w = np.dot(Ainv, b-lambda_)
    sum_ = np.sum(w.flatten())
    J = (sum_ - 1)**2
    v = 1000 * np.maximum(0,np.abs(w - 1/2).flatten() - 1/2)
    v = v ** 2
    J += np.sum(v)
    return J

def reslinreg(params, mode = 'mse'):
    X_slice, y_slice = params['X_slice'], params['y_slice']
    X_yr, y_yr = params['X_yr'], params['y_yr']
    
    err_tr_reslinreg, err_dev_reslinreg = [], []
    max_entry = {} 

    for year in range(19,23):
        X_train, y_train = X_slice[year], y_slice[year]
        X_dev, y_dev = X_yr[year], y_yr[year]
        
        A = 2 * np.dot(X_train.T, X_train)
        Ainv = inv(A)
        b = 2 * np.dot(X_train.T, y_train) # setting Ainv and b to optimize fun()

        lambda_ = 1
        min_ = minimize(fun, 1, args = (Ainv, b)) #fixing parameters Ainv and b
        lambda_ = min_.x

        x = b - lambda_
        w = np.dot(Ainv, b-lambda_)
 
        y_dev_pred = np.dot(X_yr[year], w)
        y_dev_pred = np.minimum(y_dev_pred, 100)
        if mode == 'mse':
            err_dev_reslinreg.append(mse(y_dev_pred, y_yr[year]))
        if mode == 'mae':
            err_dev_reslinreg.append(mae(y_dev_pred, y_yr[year]))
            
    
    if mode == 'mse':
            outcomes = [int(x) for x in err_dev_reslinreg] + [int(np.max(err_dev_reslinreg))]
    if mode == 'mae':
        outcomes = [int(x) for x in err_dev_reslinreg] + [int(np.mean(err_dev_reslinreg))]
    
    return outcomes

In [4]:
def linreg(params, X_true = X.copy(), y_yr_true = {}, type_ = 'normal', distr = {}, mode = 'mse'):
    X_slice_scaled, y_slice = params['X_slice_scaled'], params['y_slice']
    X_yr_scaled, y_yr = params['X_yr_scaled'], params['y_yr']
    
    if y_yr_true == {}:
        y_yr_true = y_yr.copy()
    err_tr_linreg, err_dev_linreg = [], []

    for year in range(19,23):
        X_train, y_train = X_slice_scaled[year], y_slice[year]
        X_dev, y_dev = X_yr_scaled[year], y_yr[year]    

        linreg = LinearRegression()
        linreg.fit(X_train, y_train)
        
        y_dev_pred = np.minimum(linreg.predict(X_dev), 100)
        
        if type_ == 'perc':
            y_dev = y_yr_true[year]
            y_dev_pred = 100 * stats.scoreatpercentile(distr[year], y_dev_pred)    
            
        if mode == 'mse':
            err_dev_linreg.append(mse(y_dev_pred, y_yr[year]))
        if mode == 'mae':
            err_dev_linreg.append(mae(y_dev_pred, y_yr[year]))
            
    
    if mode == 'mse':
            outcomes = [int(x) for x in err_dev_linreg] + [int(np.max(err_dev_linreg))]
    if mode == 'mae':
        outcomes = [int(x) for x in err_dev_linreg] + [int(np.mean(err_dev_linreg))]
    return outcomes
    

In [5]:
def best_tree(params, y_yr_true = {}, type_ = 'normal', distr = {}, 
              max_depth_best = 0, min_samples_leaf_best = 0, mode = 'mse'):
    X_slice, y_slice = params['X_slice'], params['y_slice']
    X_yr, y_yr = params['X_yr'], params['y_yr']    
    
    if y_yr_true == {}:
        y_yr_true = y_yr.copy()
    err_dev_max_best = 10000 #best cross-val error, i.e. average over single dev errors
    err_dev_best = [] #list of dev errors that give the best average

    #finding best params for max_depth and min_samples_leaf
    if mode == 'mse':
        for max_depth in range(1,6):
            for min_samples_leaf in range(1,4):
                tree = DecisionTreeRegressor(max_depth = max_depth, min_samples_leaf = min_samples_leaf)

                err_train_temp = []
                err_dev_temp = []

                for year in range(19,23):
                    X_train = X_slice[year]
                    y_train = y_slice[year]
                    X_dev = X_yr[year]
                    y_dev = y_yr[year]

                    tree.fit(X_train, y_train)
                    y_dev_pred = tree.predict(X_dev)

                    if type_ == 'perc':
                        y_dev = y_yr_true[year]
                        y_dev_pred = 100 * stats.scoreatpercentile(distr[year], y_dev_pred)                    

                    err_dev_temp.append(mse(y_dev_pred, y_dev))

                err_dev_max_temp = np.max(err_dev_temp)

                if err_dev_max_best > err_dev_max_temp:

                    err_dev_max_best = err_dev_max_temp
                    err_dev_best = err_dev_temp.copy()
                    max_depth_best = max_depth
                    min_samples_leaf_best = min_samples_leaf
    
    err_dev_best = []
    #finding predictions for the best tree
    for year in range(19,23):
        tree = DecisionTreeRegressor(max_depth = max_depth_best, min_samples_leaf = min_samples_leaf_best)
        X_train = X_slice[year]
        y_train = y_slice[year]
        X_dev = X_yr[year]
        y_dev = y_yr[year]

        tree.fit(X_train, y_train)
        y_dev_pred = tree.predict(X_dev)

        if type_ == 'perc':
            y_dev = y_yr_true[year]
            y_dev_pred = 100 * stats.scoreatpercentile(distr[year], y_dev_pred) 
            
        if mode == 'mse':
            err_dev_best.append(mse(y_dev_pred, y_yr[year]))
        if mode == 'mae':
            err_dev_best.append(mae(y_dev_pred, y_yr[year]))
            
    
    if mode == 'mse':
        outcomes = [int(x) for x in err_dev_best] + [int(np.max(err_dev_best))]
        print('Best max depth for tree: ' + str(max_depth_best))
        print('Best min_samples_leaf for tree: ' + str(min_samples_leaf) + '\n')
    if mode == 'mae':
        outcomes = [int(x) for x in err_dev_best] + [int(np.mean(err_dev_best))]
        
    
    
    return outcomes, max_depth_best, min_samples_leaf_best
    

In [6]:
#Setting max_depth = 1 to avoid overfitting to dev set
def best_forest(params, max_depth = 1, y_yr_true = {}, type_ = 'normal', distr = {}, mode = 'mse'): 
    
    X_slice_trimmed_scaled, y_slice = params['X_slice_trimmed_scaled'], params['y_slice']
    X_yr_trimmed_scaled, y_yr = params['X_yr_trimmed_scaled'], params['y_yr']
    
    
    if y_yr_true == {}:
        y_yr_true = y_yr.copy()
    err_dev_best, err_dev_temp = {}, {}
    err_dev_max_best = 10000
    max_depth_best = 0
    n_estimators_best = 0

    for n_estimators in range(1,20):
        forest = RandomForestRegressor(n_estimators = n_estimators, max_depth = max_depth, random_state = 42)
        for year in range(19,23):
            X_train, y_train = X_slice_trimmed_scaled[year], y_slice[year] #selecting only significant cols
            X_dev, y_dev = X_yr_trimmed_scaled[year], y_yr[year]   

            forest.fit(X_train, y_train)    
            y_dev_pred = forest.predict(X_dev)
            
            if type_ == 'perc':
                y_dev = y_yr_true[year]
                y_dev_pred = 100 * stats.scoreatpercentile(distr[year], y_dev_pred)  

            err_dev_temp[year] = mse(y_dev_pred, y_dev)

        err_dev_max_temp = np.max(list(err_dev_temp.values()))
        
        if err_dev_max_best > err_dev_max_temp:  
            
            err_dev_max_best = err_dev_max_temp
            max_depth_best = max_depth
            n_estimators_best = n_estimators
            
    
    forest = RandomForestRegressor(n_estimators = n_estimators_best, max_depth = max_depth, random_state = 24)
    for year in range(19,23):
        X_train, y_train = X_slice_trimmed_scaled[year], y_slice[year] 
        X_dev, y_dev = X_yr_trimmed_scaled[year], y_yr[year]   

        forest.fit(X_train, y_train)    
        y_dev_pred = forest.predict(X_dev)
        
        if type_ == 'perc':
            y_dev = y_yr_true[year]
            y_dev_pred = 100 * stats.scoreatpercentile(distr[year], y_dev_pred)  

            
        if mode == 'mse':
            err_dev_best[year] = mse(y_dev_pred, y_dev)
        if mode == 'mae':
            err_dev_best[year] = mae(y_dev_pred, y_dev)
            
    
    if mode == 'mse':
        err_dev_max_best = np.max(list(err_dev_best.values()))
        outcomes = [int(x) for x in list(err_dev_best.values())] + [int(err_dev_max_best)]
    if mode == 'mae':
        err_dev_mean_best = np.mean(list(err_dev_best.values()))
        outcomes = [x for x in list(err_dev_best.values())] + [err_dev_mean_best]
        

    
            
            
    print('Best max depth for forest: ' + str(max_depth_best))
    print('Best number of estimators: ' + str(n_estimators_best) + '\n')
    return outcomes


In [7]:
#Setting max_depth = 1 to avoid overfitting to dev set
def best_xgb(params, max_depth = 1, y_yr_true = {}, type_ = 'normal', distr = {}, mode = 'mse'):
    
    X_slice_trimmed_scaled, y_slice = params['X_slice_trimmed_scaled'], params['y_slice']
    X_yr_trimmed_scaled, y_yr = params['X_yr_trimmed_scaled'], params['y_yr']
    
    if y_yr_true == {}:
        y_yr_true = y_yr.copy()
    err_dev_best = {}
    err_dev_temp = {}
    err_dev_max_best = 10000
    max_depth_best = 0
    n_estimators_best = 0


    for n_estimators in range(1,21):
        
        xgb = XGBRegressor(n_estimators = n_estimators, max_depth = max_depth, random_state = 42)
        for year in range(19,23):
            X_train, y_train = X_slice_trimmed_scaled[year], y_slice[year] #selecting only significant cols
            X_dev, y_dev = X_yr_trimmed_scaled[year], y_yr[year] 

            xgb.fit(X_train, y_train)    
            y_dev_pred = np.minimum(xgb.predict(X_dev), 100)
            
            if type_ == 'perc':
                
                y_dev = y_yr_true[year]
                y_dev_pred = 100 * stats.scoreatpercentile(distr[year], y_dev_pred)  

            err_dev_temp[year] = mse(y_dev_pred, y_dev)

        err_dev_max_temp = np.max(list(err_dev_temp.values()))
        if err_dev_max_best > err_dev_max_temp:
            
            err_dev_max_best = err_dev_max_temp
            n_estimators_best = n_estimators
    
    #Running the best n_estimators with different random_state to get less optimistic errors
    xgb = XGBRegressor(n_estimators = n_estimators_best, max_depth = max_depth, random_state = 24)
    for year in range(19,23):
        X_train, y_train = X_slice_trimmed_scaled[year], y_slice[year] 
        X_dev, y_dev = X_yr_trimmed_scaled[year], y_yr[year]   

        xgb.fit(X_train, y_train)    
        y_dev_pred = np.minimum(xgb.predict(X_dev), 100)
        
        if type_ == 'perc':
            y_dev = y_yr_true[year]
            y_dev_pred = 100 * stats.scoreatpercentile(distr[year], y_dev_pred)  

        
        if mode == 'mse':
            err_dev_best[year] = mse(y_dev_pred, y_dev)
        if mode == 'mae':
            err_dev_best[year] = mae(y_dev_pred, y_dev)
            
    
    if mode == 'mse':
        err_dev_max_best = np.max(list(err_dev_best.values()))
        outcomes = [int(x) for x in list(err_dev_best.values())] + [int(err_dev_max_best)]
    if mode == 'mae':
        err_dev_mean_best = np.mean(list(err_dev_best.values()))
        outcomes = [x for x in list(err_dev_best.values())] + [err_dev_mean_best]
            
    
    print('Best max depth for XGBoost: ' + str(max_depth))
    print('Best number of estimators: ' + str(n_estimators_best) + '\n')
    return outcomes
    

### 1. Normal data

In [8]:
params = initialize_cv_split(df, X, y)

reslinreg_data = reslinreg(params)
linreg_data = linreg(params)
tree_data, max_depth_best, min_leaf_samples_leaf = best_tree(params)
forest_data = best_forest(params, max_depth_best)
xgb_data = best_xgb(params, max_depth_best)

data ={}
col_names = ['Name', 19, 20, 21, 22, 'total']

data = [['Restricted Linear Regression'] + reslinreg_data, 
        ['Linear Regression'] + linreg_data, ['Decision tree'] + tree_data, 
        ['Random forest'] + forest_data, ['XGBoost'] + xgb_data]

models = pd.DataFrame(data, columns = col_names)

models

Best max depth for tree: 1
Best min_samples_leaf for tree: 3

Best max depth for forest: 1
Best number of estimators: 10

Best max depth for XGBoost: 1
Best number of estimators: 9



,Name,19,20,21,22,total
0,Restricted Linear Regression,382,151,108,433,433
1,Linear Regression,534,134,133,364,534
2,Decision tree,507,241,282,387,507
3,Random forest,450,160,207,266,450
4,XGBoost,354,135,158,368,368


Okay, we see that the above functions reproduce what we got in Basic_models.ipynb. Now let's transform our data a bit and see what happens.

### 2. % $\to$ Percentile $\to$ % mapping

Let's now try to map our data from percentages to percentiles, since they seem to be more informative than normal percentages, e.g. they include the information that the results usually are really ''dense'' around the score of 40%. 

Since our aim is to finally output percentages, this is what we'll do:

On each training, we hide one of the years of data, map our training data (i.e. everything but one year) according to the distribution, which is the sum of the distributions of exams from these 3 training years (unfortunately, we don't have any other info about distributions except for the final exams), map the training data into percentiles, then find the best models to output desired percentiles and then finally map the percentiles back to the percentage scores.

In [9]:
# inputting exam distribution data

data = {}
sum_ = {}
exam = {}

data[19] = np.array([0.15, 0.9, 2.3, 3.9, 4.95, 5.4, 5.25, 5, 4.8, 4.55, 4.5, 4.25, 4.1, 3.9, 3.75, 3.6, 3.45, 3.25, 3.15, 3.1,
                2.95, 2.8, 2.7, 2.65, 2.55, 2.45, 2.3, 2.1, 1.85, 1.6, 1.35])
data[20] = np.array([0.05, 0.3, 1.1, 2.35, 3.7, 4.8, 5.4, 5.7, 5.55, 5.25, 5, 4.7, 4.5, 4.3, 4.1, 4, 3.8, 3.6, 3.5, 3.4, 3.2, 3.05,
                    2.95, 2.7, 2.5, 2.3, 2.1, 1.9, 1.65, 1.1, 1.35])
data[21] = np.array([0.1, 0.6, 2, 4.1, 6.1, 6.9, 6.7, 6.15, 5.45, 5.15, 4.95, 4.75, 4.55, 4.3, 4.15, 3.95, 3.75, 3.6, 3.35, 3.3, 3.05, 2.9, 2.8, 2.65, 2.55, 2.3])
data[22] = np.array([0.15, 0.75, 2, 3.5, 4.5, 4.8, 4.6, 4.3, 3.95, 3.7, 3.55, 3.45, 3.4, 3.3, 3.25, 3.35, 3.45, 3.5, 3.7, 3.95, 4.35, 4.75, 5.45, 6.2, 6.8, 5.35])

for year in range(19,23):
    sum_[year] = np.sum(data[year])
    data[year] = data[year]  / sum_[year]
    exam[year] = {'perc': data[year]}
    exam[year] = pd.DataFrame(exam[year])

In [10]:
# Data that simulates our distributions
sim_data, sim_data_slice = {}, {} #
for year in range(19,23):
    sim_data[year] = []
    for i in range(len(data[year])):
        for j in range(int(1000 * data[year][i] + 0.01)):
            sim_data[year].append(i/(len(data[year])-1))
            
# Data that simulates ''slice'' distributions, i.e. joint distributions of all years but one.
for year in range(19,23):
    sim_data_slice[year] = []
    for yr in range(19,23):
        if yr != year:
            sim_data_slice[year] += sim_data[yr]
    sim_data_slice[year].sort()


Now that we have our data simulation, one important note. We want to transform our data percentages into percentiles differently for our train set and dev set.

Namely, when it comes to our training data (X_slice/X_slice_trimmed), we want it to be mapped to percentiles using distributions from the years of our training data, whereas our dev data (X_yr/X_yr_trimmed) should be mapped using distributions of all years but the year of our dev data (otherwise, we would pass it unknown informations about the general distributions of the year that we want to predict).

In [11]:
# Mapping data to percentiles

# Getting the usual data
params = initialize_cv_split(df, X, y)

X_yr, y_yr = params['X_yr'], params['y_yr'] 
X_slice, y_slice = params['X_slice'], params['y_slice'] 
X_slice_trimmed, X_yr_trimmed = params['X_slice_trimmed'], params['X_yr_trimmed'] 
X_slice_scaled, X_yr_scaled, X_slice_trimmed_scaled, X_yr_trimmed_scaled = \
params['X_slice_scaled'], params['X_yr_scaled'], params['X_slice_trimmed_scaled'], params['X_yr_trimmed_scaled']

# Changing training data according to their distributions.
X_slice_perc, X_yr_perc = {}, {} #Percentile analogues of the data above
y_slice_perc, y_yr_perc = {}, {}
X_slice_trimmed_perc, X_yr_trimmed_perc = {}, {} #Percentile analogues of the data above
X_slice_scaled_perc, X_yr_scaled_perc, X_slice_trimmed_scaled_perc, X_yr_trimmed_scaled_perc = {}, {}, {}, {} 

X_yr_true_perc = {} 
#Percentiles of scores for the given year mapped using this years distribution, these will form building blocks for X_slice


scaler = StandardScaler()

for year in range(19,23):
    X_yr_perc[year], X_yr_true_perc[year] = X_yr[year].copy(), X_yr[year].copy()
    y_yr_perc[year], y_slice_perc[year] = y_yr[year].copy(), y_slice[year].copy()
    for col in X.columns:
        # Making percentile versions of X_yr and X_slice
        X_yr_perc[year][col] = stats.percentileofscore(sim_data_slice[year], X_yr_perc[year][col] / 100)
        X_yr_true_perc[year][col] = stats.percentileofscore(sim_data[year], X_yr_true_perc[year][col] / 100)

X_true_perc = pd.concat([X_yr_true_perc[19], X_yr_true_perc[20], X_yr_true_perc[21], X_yr_true_perc[22]], ignore_index=True)

for year in range(19,23):
    X_slice_perc[year] = X_true_perc[df['rocznik'] != year]
        
for year in range(19,23):
    #Making percentile versions of y_yr and y_slice
    y_yr_perc[year] = stats.percentileofscore(sim_data[year], y_yr_perc[year] / 100)
    y_slice_perc[year] = stats.percentileofscore(sim_data_slice[year], y_slice_perc[year] / 100)
        
    # Making percentile versions of X_yr_trimmed, X_slice_trimmed
    X_yr_trimmed_perc[year] = X_yr_perc[year].iloc[:,[0,4,5]]
    X_slice_trimmed_perc[year] = X_slice_perc[year].iloc[:,[0,4,5]]

    # Making percentile versions of X_..._scaled
    X_slice_scaled_perc[year] = scaler.fit_transform(X_slice_perc[year])
    X_yr_scaled_perc[year] = scaler.transform(X_yr_perc[year])
    X_slice_trimmed_scaled_perc[year] = scaler.fit_transform(X_slice_trimmed_perc[year])
    X_yr_trimmed_scaled_perc[year] = scaler.transform(X_yr_trimmed_perc[year])
        
params['X_slice'], params['X_yr'] = X_slice_perc, X_yr_perc 
params['X_slice_trimmed'], params['X_yr_trimmed'] = X_slice_trimmed_perc, X_yr_trimmed_perc
params['X_slice_scaled'], params['X_yr_scaled'], params['X_slice_trimmed_scaled'], params['X_yr_trimmed_scaled'] = \
X_slice_scaled_perc, X_yr_scaled_perc, X_slice_trimmed_scaled_perc, X_yr_trimmed_scaled_perc


In [12]:
# Testing our models against percentile scores

linreg_data = linreg(params, y_yr_true =  params['y_yr'].copy(), type_ = 'perc', distr = sim_data_slice)
tree_data, max_depth_best, _ = best_tree(params, y_yr_true =  params['y_yr'].copy(), type_ = 'perc', distr = sim_data_slice)
forest_data = best_forest(params, y_yr_true =  params['y_yr'].copy(), type_ = 'perc', distr = sim_data_slice)
xgb_data = best_xgb(params, y_yr_true =  params['y_yr'].copy(), type_ = 'perc', distr = sim_data_slice)

data_perc ={}
col_names = ['Name', 19, 20, 21, 22, 'total']

data_perc = [['Linear Regression'] + linreg_data, ['Decision tree'] + tree_data, 
        ['Random forest'] + forest_data, ['XGBoost'] + xgb_data]

models_perc = pd.DataFrame(data_perc, columns = col_names)
models_perc

Best max depth for tree: 4
Best min_samples_leaf for tree: 3

Best max depth for forest: 1
Best number of estimators: 2

Best max depth for XGBoost: 1
Best number of estimators: 19



,Name,19,20,21,22,total
0,Linear Regression,747,217,254,674,747
1,Decision tree,567,334,339,749,749
2,Random forest,569,195,193,692,692
3,XGBoost,513,187,144,520,520


In [13]:
# Exporting the data to the main notebook

models_perc.to_excel('score_perc_score.xlsx')

### 3. Percentile $\to$ percentile mapping

In [14]:
reslinreg_data = reslinreg(params)
linreg_data = linreg(params)
tree_data, max_depth_best, _ = best_tree(params)
forest_data = best_forest(params)
xgb_data = best_xgb(params)

data2 ={}
col_names = ['Name', 19, 20, 21, 22, 'total']

data2 = [['Restricted Linear Regression'] + reslinreg_data, 
        ['Linear Regression'] + linreg_data, ['Decision tree'] + tree_data, 
        ['Random forest'] + forest_data, ['XGBoost'] + xgb_data]

models2 = pd.DataFrame(data2, columns = col_names)

models2

Best max depth for tree: 5
Best min_samples_leaf for tree: 3

Best max depth for forest: 1
Best number of estimators: 2

Best max depth for XGBoost: 1
Best number of estimators: 8



,Name,19,20,21,22,total
0,Restricted Linear Regression,761,174,196,396,761
1,Linear Regression,739,228,272,360,739
2,Decision tree,635,369,313,442,635
3,Random forest,579,188,194,395,579
4,XGBoost,409,158,164,415,415


In [15]:
# Exporting the data to the main notebook

models2.to_excel('perc_to_perc.xlsx')

### 4. Data augmentation - making ''shadows'' of students

Note that in the normal model scores $\to$ scores, we don't really take into account the information that we have about the distributions of the exams. 
But how could we possibly include in our dataset/training the information about the distribution other than by scores $\to$ percentile mapping as before?

Well, we can augment additional data, by creating a ''shadow'' of each student, i.e. for student $A$ make an instance of student $A'$ so that for each test if $A$ scored $s$ points getting to $p$-th percentile, $A'$ ''scored'' $s'$ points, 
which places them on $100-p$-th percentile.

Let's try this idea below but this time, since we are not in the need of percentile mapping of our dev set, let's make each X_slice percentile mapped purely by the distribution of their final exam distribution, not the sum of the three years that are in the slice.

In [16]:
# Mapping df into percentiles according to end exams results.

df_yr, df_perc_yr = {}, {}
for year in range(19,23):
    df_yr[year] = df[df['rocznik'] == year]

for year in range(19,23):
    df_perc_yr[year] = df_yr[year].copy()
    for col in df_yr[year].columns[1:]:
        df_perc_yr[year][col] = stats.percentileofscore(sim_data[year], df_perc_yr[year][col] / 100)
        
df_perc = pd.concat([df_perc_yr[19], df_perc_yr[20], df_perc_yr[21], df_perc_yr[22]], ignore_index = True)


In [17]:
# Making ''inverses'' of students
df_perc_inv = df_perc.copy()
df_perc_inv.iloc[:,1:] = 100 - df_perc_inv.iloc[:,1:]

# Mapping ''inverses'' into scores
df_perc_inv_yr, df_inv_yr = {}, {}
for year in range(19,23):
    df_perc_inv_yr[year] = df_perc_inv[df_perc_inv['rocznik'] == year]
    
for year in range(19,23):
    df_inv_yr[year] = df_perc_inv_yr[year].copy()
    for col in df_perc_inv_yr[year].columns[1:]:
        df_inv_yr[year][col] = 100 * stats.scoreatpercentile(sim_data[year], df_perc_inv_yr[year][col])

df_inv = pd.concat([df_inv_yr[19], df_inv_yr[20], df_inv_yr[21], df_inv_yr[22]], ignore_index = True)

display(df.head())
df_inv.head()

,rocznik,sesja I,X,XII,sesja II,II,III,wynik
0,19,77.0,53.0,63.0,75.0,75.0,61.0,60
1,19,85.0,69.0,97.0,93.0,91.0,88.0,93
2,19,41.0,31.0,33.0,48.0,66.0,36.0,50
3,19,78.0,81.0,83.0,70.0,94.0,91.0,93
4,19,26.0,22.0,53.0,25.0,41.0,21.0,27


,rocznik,sesja I,X,XII,sesja II,II,III,wynik
0,19,16.666667,33.333333,26.666667,16.666667,16.666667,26.666667,26.666667
1,19,13.333333,20.000000,6.666667,10.000000,10.000000,10.000000,10.000000
2,19,40.000000,53.333333,53.333333,33.333333,23.333333,46.666667,33.333333
3,19,16.666667,13.333333,13.333333,20.000000,6.666667,10.000000,10.000000
4,19,63.333333,66.666667,33.333333,63.333333,40.000000,66.666667,56.666667


Ok, let's now test it on our models.

In [18]:
params = initialize_cv_split(df, X, y)

# Why didn't we pass df_inv to initialize_cv_split? We want df_inv only to be in our training set but not in dev set
scaler = StandardScaler()
for year in range(19,23):
    params['X_slice'][year] = pd.concat([params['X_slice'][year], 
                                         df_inv[df_inv['rocznik'] != year].iloc[:,1:-1]], ignore_index= True)
    params['y_slice'][year] = pd.concat([params['y_slice'][year], 
                                         df_inv[df_inv['rocznik'] != year]['wynik']], ignore_index= True)
    
    params['X_slice_trimmed'][year], params['X_yr_trimmed'][year] = \
    params['X_slice'][year].iloc[:,[0,4,5]], params['X_yr'][year].iloc[:,[0,4,5]]
    
    #scaling normal data
    params['X_slice_scaled'][year] = scaler.fit_transform(params['X_slice'][year])
    params['X_yr_scaled'][year] = scaler.transform(params['X_yr'][year])

    #scaling trimmed versions
    params['X_slice_trimmed_scaled'][year] = scaler.fit_transform(params['X_slice_trimmed'][year])
    params['X_yr_trimmed_scaled'][year] = scaler.transform(params['X_yr_trimmed'][year])


reslinreg_data = reslinreg(params)
linreg_data = linreg(params)
tree_data, max_depth_best, min_samples_leaf_best = best_tree(params)
forest_data = best_forest(params)
xgb_data = best_xgb(params)

data ={}
col_names = ['Name', 19, 20, 21, 22, 'total']

data_aug = [['Restricted Linear Regression'] + reslinreg_data, 
        ['Linear Regression'] + linreg_data, ['Decision tree'] + tree_data, 
        ['Random forest'] + forest_data, ['XGBoost'] + xgb_data]

models_aug = pd.DataFrame(data_aug, columns = col_names)

models_aug

Best max depth for tree: 3
Best min_samples_leaf for tree: 3

Best max depth for forest: 1
Best number of estimators: 17

Best max depth for XGBoost: 1
Best number of estimators: 19



,Name,19,20,21,22,total
0,Restricted Linear Regression,669,238,441,139,669
1,Linear Regression,366,288,249,488,488
2,Decision tree,483,394,447,562,562
3,Random forest,264,494,239,438,494
4,XGBoost,246,348,215,334,348


In [19]:
# Exporting the data to the main notebook

models_aug.to_excel('model_aug.xlsx')

### 5. More data augmentation - making "translations" of students

As described in the write-up, now instead of making "shadows" of students, we will make their translations, i.e. 
having set fixed $\hat{p}$ we take each students scores, map them to percentiles $p$ (based on the year's 
exam distributions), translate them to percentiles $p+\hat{p}$ and then back to scores.

Details: we will just consider $\hat{p}\in\{-60,-40,\ldots, 40,60\}$ and 
translate the student's percentiles only if they don't fall out of bounds ([0,100]).
On one hand, this will make different number of copies of each students but note that this way we 
give smaller "weight" to the students that have big variance of results and hence are hard to cope with.

In [20]:
# Mapping our data to percentiles and splitting into separate years.
# Mapping df into percentiles according to end exams results.

df_yr, df_trans_yr = {}, {}
for year in range(19,23):
    df_yr[year] = df[df['rocznik'] == year]

for year in range(19,23):
    df_trans_yr[year] = df_yr[year].copy()
    for col in df_yr[year].columns[1:]:
        df_trans_yr[year][col] = stats.percentileofscore(sim_data[year], df_trans_yr[year][col] / 100)

for year in range(19,20):
    for p_hat in np.arange(-20, 21, 20):
        if p_hat != 0:
            df_trans_temp = df_trans_yr[year].copy()
            df_trans_temp.iloc[:,1:] = df_trans_temp.iloc[:,1:] + p_hat

            # Adding min and max columns
            min_ = np.min(df_trans_temp.iloc[:,1:], axis = 1)
            df_trans_temp['min'] = min_
            max_ = np.max(df_trans_temp.iloc[:,1:], axis = 1)
            df_trans_temp['max'] = max_

            # Checking out-of-boundedness
            df_trans_temp = df_trans_temp[df_trans_temp['min'] > 0]
            df_trans_temp = df_trans_temp[df_trans_temp['max'] < 100]
            df_trans_temp = df_trans_temp.drop(columns = ['min', 'max'])

            # Mapping back
            for col in df_trans_temp.columns[1:]:
                df_trans_temp[col] = 100 * stats.scoreatpercentile(sim_data[year], df_trans_temp[col])

            #Adding df_trans_temp to df_trans_yr
            df_trans_yr[year] = pd.concat([df_trans_yr[year], df_trans_temp], ignore_index=True)
        
# Making a complete DataFrame out of it
df_trans = pd.concat([df_trans_yr[19], df_trans_yr[20], df_trans_yr[21], df_trans_yr[22]], ignore_index=True)
df_trans.shape

(123, 8)

In [21]:
params = initialize_cv_split(df, X, y)

# Why didn't we pass df_inv to initialize_cv_split? We want df_inv only to be in our training set but not in dev set
scaler = StandardScaler()
for year in range(19,23):
    params['X_slice'][year] = pd.concat([params['X_slice'][year], 
                                         df_trans[df_trans['rocznik'] != year].iloc[:,1:-1]], ignore_index= True)
    params['y_slice'][year] = pd.concat([params['y_slice'][year], 
                                         df_trans[df_trans['rocznik'] != year]['wynik']], ignore_index= True)
    
    params['X_slice_trimmed'][year], params['X_yr_trimmed'][year] = \
    params['X_slice'][year].iloc[:,[0,4,5]], params['X_yr'][year].iloc[:,[0,4,5]]
    
    #scaling normal data
    params['X_slice_scaled'][year] = scaler.fit_transform(params['X_slice'][year])
    params['X_yr_scaled'][year] = scaler.transform(params['X_yr'][year])

    #scaling trimmed versions
    params['X_slice_trimmed_scaled'][year] = scaler.fit_transform(params['X_slice_trimmed'][year])
    params['X_yr_trimmed_scaled'][year] = scaler.transform(params['X_yr_trimmed'][year])


reslinreg_data = reslinreg(params)
linreg_data = linreg(params)
tree_data, max_depth_best, _ = best_tree(params)
forest_data = best_forest(params)
xgb_data = best_xgb(params)

data ={}
col_names = ['Name', 19, 20, 21, 22, 'total']

data_trans = [['Restricted Linear Regression'] + reslinreg_data, 
        ['Linear Regression'] + linreg_data, ['Decision tree'] + tree_data, 
        ['Random forest'] + forest_data, ['XGBoost'] + xgb_data]

models_trans = pd.DataFrame(data_trans, columns = col_names)

models_trans

Best max depth for tree: 4
Best min_samples_leaf for tree: 3

Best max depth for forest: 1
Best number of estimators: 2

Best max depth for XGBoost: 1
Best number of estimators: 9



,Name,19,20,21,22,total
0,Restricted Linear Regression,444,147,109,477,477
1,Linear Regression,522,144,130,432,522
2,Decision tree,489,404,200,448,489
3,Random forest,446,118,168,276,446
4,XGBoost,367,121,140,369,369


(By hand inspection, out of ranges np.arange(-20, 21, 20), np.arange(-40, 41, 20), np.arange(-60, 61, 20), np.arange(-80, 81, 20), the best is the first one.)

In [22]:
# Exporting the data to the main notebook

models_trans.to_excel('models_trans.xlsx')

### 7. Shadows and translations

Essentially, it suffices to pass our "translations" dataframe into "shadows" code and we should be done. Here we go:

In [23]:
# Mapping our data to percentiles and splitting into separate years.
# Mapping df into percentiles according to end exams results.

df_yr, df_trans_yr = {}, {}
for year in range(19,23):
    df_yr[year] = df[df['rocznik'] == year]

for year in range(19,23):
    df_trans_yr[year] = df_yr[year].copy()
    for col in df_yr[year].columns[1:]:
        df_trans_yr[year][col] = stats.percentileofscore(sim_data[year], df_trans_yr[year][col] / 100)

for year in range(19,20):
    for p_hat in np.arange(-20, 21, 20):
        if p_hat != 0:
            df_trans_temp = df_trans_yr[year].copy()
            df_trans_temp.iloc[:,1:] = df_trans_temp.iloc[:,1:] + p_hat

            # Adding min and max columns
            min_ = np.min(df_trans_temp.iloc[:,1:], axis = 1)
            df_trans_temp['min'] = min_
            max_ = np.max(df_trans_temp.iloc[:,1:], axis = 1)
            df_trans_temp['max'] = max_

            # Checking out-of-boundedness
            df_trans_temp = df_trans_temp[df_trans_temp['min'] > 0]
            df_trans_temp = df_trans_temp[df_trans_temp['max'] < 100]
            df_trans_temp = df_trans_temp.drop(columns = ['min', 'max'])

            # Mapping back
            for col in df_trans_temp.columns[1:]:
                df_trans_temp[col] = 100 * stats.scoreatpercentile(sim_data[year], df_trans_temp[col])

            #Adding df_trans_temp to df_trans_yr
            df_trans_yr[year] = pd.concat([df_trans_yr[year], df_trans_temp], ignore_index=True)
        
# Making a complete DataFrame out of it
df_trans = pd.concat([df_trans_yr[19], df_trans_yr[20], df_trans_yr[21], df_trans_yr[22]], ignore_index=True)
X_trans = df_trans.iloc[:,1:-1]
y_trans = df_trans['wynik']

df_trans.shape

(123, 8)

In [24]:
# Mapping df into percentiles according to end exams results.
df_trans
df_trans_yr, df_trans_perc_yr = {}, {}
for year in range(19,23):
    df_trans_yr[year] = df_trans[df_trans['rocznik'] == year]

for year in range(19,23):
    df_trans_perc_yr[year] = df_trans_yr[year].copy()
    for col in df_trans_yr[year].columns[1:]:
        df_trans_perc_yr[year][col] = stats.percentileofscore(sim_data[year], df_trans_perc_yr[year][col] / 100)
        
df_trans_perc = pd.concat([df_trans_perc_yr[19], df_trans_perc_yr[20], 
                           df_trans_perc_yr[21], df_trans_perc_yr[22]], ignore_index = True)


In [25]:
# Making ''inverses'' of students
df_trans_perc_inv = df_trans_perc.copy()
df_trans_perc_inv.iloc[:,1:] = 100 - df_trans_perc_inv.iloc[:,1:]

# Mapping ''inverses'' into scores
df_trans_perc_inv_yr, df_trans_inv_yr = {}, {}
for year in range(19,23):
    df_trans_perc_inv_yr[year] = df_trans_perc_inv[df_trans_perc_inv['rocznik'] == year]
    
for year in range(19,23):
    df_trans_inv_yr[year] = df_trans_perc_inv_yr[year].copy()
    for col in df_trans_perc_inv_yr[year].columns[1:]:
        df_trans_inv_yr[year][col] = 100 * stats.scoreatpercentile(sim_data[year], df_trans_perc_inv_yr[year][col])

df_trans_inv = pd.concat([df_trans_inv_yr[19], df_trans_inv_yr[20], 
                          df_trans_inv_yr[21], df_trans_inv_yr[22]], ignore_index = True)

display(df_trans_inv.shape)

(123, 8)

In [26]:
params = initialize_cv_split(df, X, y)

# Why didn't we pass df_inv to initialize_cv_split? We want df_inv only to be in our training set but not in dev set
scaler = StandardScaler()
for year in range(19,23):
    params['X_slice'][year] = pd.concat([params['X_slice'][year], 
                                         df_trans[df_trans['rocznik'] != year].iloc[:,1:-1]], ignore_index= True)
    params['X_slice'][year] = pd.concat([params['X_slice'][year], 
                                         df_trans_inv[df_trans_inv['rocznik'] != year].iloc[:,1:-1]], ignore_index= True)
    params['y_slice'][year] = pd.concat([params['y_slice'][year], 
                                         df_trans[df_trans['rocznik'] != year]['wynik']], ignore_index= True)
    params['y_slice'][year] = pd.concat([params['y_slice'][year], 
                                         df_trans_inv[df_trans_inv['rocznik'] != year]['wynik']], ignore_index= True)
    
    params['X_slice_trimmed'][year], params['X_yr_trimmed'][year] = \
    params['X_slice'][year].iloc[:,[0,4,5]], params['X_yr'][year].iloc[:,[0,4,5]]
    
    #scaling normal data
    params['X_slice_scaled'][year] = scaler.fit_transform(params['X_slice'][year])
    params['X_yr_scaled'][year] = scaler.transform(params['X_yr'][year])

    #scaling trimmed versions
    params['X_slice_trimmed_scaled'][year] = scaler.fit_transform(params['X_slice_trimmed'][year])
    params['X_yr_trimmed_scaled'][year] = scaler.transform(params['X_yr_trimmed'][year])


reslinreg_data = reslinreg(params)
linreg_data = linreg(params)
tree_data, max_depth_best, _ = best_tree(params)
forest_data = best_forest(params)
xgb_data = best_xgb(params)

data ={}
col_names = ['Name', 19, 20, 21, 22, 'total']

data_trans_aug = [['Restricted Linear Regression'] + reslinreg_data, 
        ['Linear Regression'] + linreg_data, ['Decision tree'] + tree_data, 
        ['Random forest'] + forest_data, ['XGBoost'] + xgb_data]

model_trans_aug = pd.DataFrame(data_trans_aug, columns = col_names)

model_trans_aug

Best max depth for tree: 2
Best min_samples_leaf for tree: 3

Best max depth for forest: 1
Best number of estimators: 2

Best max depth for XGBoost: 1
Best number of estimators: 20



,Name,19,20,21,22,total
0,Restricted Linear Regression,750,309,224,347,750
1,Linear Regression,347,280,244,486,486
2,Decision tree,360,308,229,407,407
3,Random forest,437,436,450,534,534
4,XGBoost,212,233,120,368,368


In [27]:
# Exporting the data to the main notebook

model_trans_aug.to_excel('model_trans_aug.xlsx')

### 8. Best model mean mae error

Here is mae of the best model, i.e. XGBoost of depth = 1 and n_estimators = 19 with augmented data by adding percentile inverses of students.

In [28]:
params = initialize_cv_split(df, X, y)

# Why didn't we pass df_inv to initialize_cv_split? We want df_inv only to be in our training set but not in dev set
scaler = StandardScaler()
for year in range(19,23):
    params['X_slice'][year] = pd.concat([params['X_slice'][year], 
                                         df_inv[df_inv['rocznik'] != year].iloc[:,1:-1]], ignore_index= True)
    params['y_slice'][year] = pd.concat([params['y_slice'][year], 
                                         df_inv[df_inv['rocznik'] != year]['wynik']], ignore_index= True)
    
    params['X_slice_trimmed'][year], params['X_yr_trimmed'][year] = \
    params['X_slice'][year].iloc[:,[0,4,5]], params['X_yr'][year].iloc[:,[0,4,5]]
    
    #scaling normal data
    params['X_slice_scaled'][year] = scaler.fit_transform(params['X_slice'][year])
    params['X_yr_scaled'][year] = scaler.transform(params['X_yr'][year])

    #scaling trimmed versions
    params['X_slice_trimmed_scaled'][year] = scaler.fit_transform(params['X_slice_trimmed'][year])
    params['X_yr_trimmed_scaled'][year] = scaler.transform(params['X_yr_trimmed'][year])

xgb_data = best_xgb(params, mode = 'mae')

data ={}
col_names = ['Name', 19, 20, 21, 22, 'total']

data_aug = [['XGBoost'] + xgb_data]

models_aug_mae = pd.DataFrame(data_aug, columns = col_names)

models_aug_mae

Best max depth for XGBoost: 1
Best number of estimators: 19



,Name,19,20,21,22,total
0,XGBoost,12.113492,13.636335,12.022682,15.499053,13.317891


In [29]:
models_aug_mae.to_excel('models_aug_mae.xlsx')